Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [X] Choose your target. Which column in your tabular dataset will you predict?
- [X] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude.
- [X] Determine whether your problem is regression or classification.
- [X] Choose your evaluation metric.
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" information from the future?


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv("https://raw.githubusercontent.com/apathyhill/DS-Unit-2-Applied-Modeling/master/data/portfolio/vgsales-12-4-2019.csv")
data2 = data.copy()

#data2["score"] = data2[["VGChartz_Score", "Critic_Score", "User_Score"]].mean(axis=1)
data2 = pd.concat([data2, pd.get_dummies(data2["ESRB_Rating"])], axis=1)

print(data2.isna().sum())
print("Columns:", list(data2.columns))
target = "Global_Sales"

train, test = train_test_split(data2, train_size=0.90)
train, val = train_test_split(train, train_size=0.75)

"""
This is a regression problem, as I am trying to output a continous number.

Simple accuracy could work as a eval matric right now, but I might change it to mean absolue/squared error to see if I can decipher things better.
"""

features_excluded = ["Last_Update", "url", "status", "Vgchartzscore", "img_url", "basename", 
                     "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales", "Rank", "Name", 
                     "VGChartz_Score", "Critic_Score", "User_Score", "ESRB_Rating"]
"""
These last columns are only for information about the game on the VGChartz website, such as cover images, and the url to the game. Not important here.
"basename" is a web-safe, simplified name of the game. (Wii Sports becomes wii-sports)
"NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales" are just  "Global_Sales" split into regions; probably not necessary.
"Rank" is not needed, as it is functionally just an index.
Each entry is a new game with its own title, so the "Name" category is *very* high cardinality, too much to use.


This leaves Genre, ESRB Rating (in seperate columns), Platform, Plublisher, Developer, and Year
"""

Rank                  0
Name                  0
basename              0
Genre                 0
ESRB_Rating       32169
Platform              0
Publisher             0
Developer            17
VGChartz_Score    55792
Critic_Score      49256
User_Score        55457
Total_Shipped     53965
Global_Sales      36377
NA_Sales          42828
PAL_Sales         42603
JP_Sales          48749
Other_Sales       40270
Year                979
Last_Update       46606
url                   0
status                0
Vgchartzscore     54993
img_url               0
AO                    0
E                     0
E10                   0
EC                    0
KA                    0
M                     0
RP                    0
T                     0
dtype: int64
Columns: ['Rank', 'Name', 'basename', 'Genre', 'ESRB_Rating', 'Platform', 'Publisher', 'Developer', 'VGChartz_Score', 'Critic_Score', 'User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales', 'Year', 'L

'\nThese last columns are only for information about the game on the VGChartz website, such as cover images, and the url to the game. Not important here.\n"basename" is a web-safe, simplified name of the game. (Wii Sports becomes wii-sports)\n"NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales" are just  "Global_Sales" split into regions; probably not necessary.\n"Rank" is not needed, as it is functionally just an index.\nEach entry is a new game with its own title, so the "Name" category is *very* high cardinality, too much to use.\n\n\nThis leaves Genre, ESRB Rating (in seperate columns), Platform, Plublisher, Developer, and Year\n'